In [17]:
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [118]:
hello_word = tokenizer("doctor", return_tensors="pt")
hi_word = tokenizer("finance", return_tensors="pt")
bye_word = tokenizer("bye", return_tensors="pt")
cat_word = tokenizer("bank", return_tensors="pt")
with torch.no_grad():
    hello_outputs = model(**hello_word)
with torch.no_grad():
    hi_outputs = model(**hi_word)
with torch.no_grad():
    bye_outputs = model(**bye_word)
with torch.no_grad():
    cat_outputs = model(**cat_word)

In [119]:
cat_emb = get_embeddings(cat_outputs)
hello_emb = get_embeddings(hello_outputs)
hi_emb = get_embeddings(hi_outputs)

print(cosine(hi_emb, hello_emb))
print(cosine(hi_emb, cat_emb))

0.5052249133586884
0.37504106760025024
